<h1 style="text-align:center; font-size: 3em;">Secret Santa Emailing</h1>
<h1 style="text-align:center; font-size: 3em;">🎅📨</h1>

<b>🕵️ Privacy First:</b> Your data stays with you. The entire process runs locally on your machine - no email storage worries !
<br><b>🔒 Secure & Reliable::</b> Uses your existing (professionnal) email account. Fully compatible with popular services like Gmail and Outlook.
<br><b>📨 User-Friendly:</b><ol>
    <li>Upload a CSV with names and addresses.</li>
    <li>Customize your email template.</li>
    <li>Preview and send each email individually with ease.</li>
</ol>
<b>🤝 Trustable & Free:</b> Fully Open Source code, but you can <a href="https://www.buymeacoffee.com/louisgeisler">Buy Me a Coffee</a>
<br><b>⚙️ Advanced Customization:</b> Thanks to <a href="https://jinja.palletsprojects.com/en/3.0.x/templates/#if">Jinja templating</a>, you can add conditional logic to your templates for a personalized touch! !
<br>

Website: [https://louisgeisler.github.io/EmailTemplatingSender/](https://louisgeisler.github.io/EmailTemplatingSender/)  
GitHub Repo: [https://github.com/louisgeisler/EmailTemplatingSender](https://github.com/louisgeisler/EmailTemplatingSender)

# Install Packages

In [ ]:
!pip install pandas
!pip install Jinja2

# Importations

In [ ]:
import io
import pandas as pd
from jinja2 import Template
from urllib.parse import quote
from IPython.core.display import HTML
from IPython.display import Javascript

# Define list of people and their informations

In [ ]:
# You may replace this by a valide csv path
csv_path = io.StringIO('''
Name,Email
Louis,louis@example.com
Ekaterina,ekaterina@example.com
Boris,boris@example.com
Elettra,elettra@example.com
Ayoub,boris@example.com
Lina,lina@example.com
''')

df = pd.read_csv(csv_path)

df

# Shuffle Emails

In [ ]:
for i in range(3):
    df = df.sample(frac=1, replace=False)

#df # Keep it secret, even from yourself... or not ;-)

# Save the sending order !

VERY IMPORTANT: In case where you need to add or delete someone

In [ ]:
df.to_csv("secret_santa_order.csv")

# Define an Email Template

In [ ]:
email_template = {
    "to": """{{ SENDER.EMAIL }}""",
    "subject": """Secret Santa 2024""",
    "body": """Hi {{ SENDER.NAME }},

For this Christmas we have plan a Secret Santa !

For this, you'll need to prepare a gift ~10€ for {{ RECEIVER.NAME }} ({{ RECEIVER.EMAIL }}) !

Please, keep the surprise until the End-Of-The-Year Party, and bring your gift for the XXth December, in XXXXXX, at XPM.

Be original, creative and see you soon ;-)


This email was automatically generated thanks to: https://github.com/louisgeisler/EmailTemplatingSender """
}

# Render Templates

Gmail need a custom way of prefil email, if you're not using gmail, write 'default' instead of Gmail

In [ ]:
mode="gmail"

In [ ]:


def enc(txt):
    return quote(txt.encode("utf-8")).replace("+", "%2B")

def email_uri(email, mode="gmail"):
    mode = mode.lower()
    if mode=="gmail":
        encoded_url = enc(f"""mailto:?to={ email['to'] }&subject={ email['subject'] }&body={ email['body'] }""");
        return f"""https://mail.google.com/mail/?extsrc=mailto&url={encoded_url}""";
    else:
        return f"""mailto:{ email['to'] }?subject={ email['subject'] }&body={ email['body'] }""";

email_templated = {
    k: Template(v)
    for k, v in email_template.items()
}

def get_row(email_i):
    return {
        k.upper(): v
        for k, v in df.iloc[email_i].to_dict().items()
    }

email_info = []
n_people = len(df)
for email_i in range(n_people):
    
    email_i_plus_1 = (email_i + 1)%n_people
    
    email_info_dict = {
        "SENDER": get_row(email_i),
        "RECEIVER": get_row(email_i_plus_1),
    }
    
    rendered_email = {
        k: template.render(
            **email_info_dict
        )
        for k, template in email_templated.items()
    }
    
    rendered_email["url"] = email_uri(rendered_email, mode)
    
    email_info += [rendered_email]

# Generate URLs

In [ ]:
html = """
<style>
a:link {
  color: green;
}
a:visited {
  color: red;
}
</style>
Send to:<br>
"""

html += "<ol>"
for info in email_info:
    html += f"""<li><a href='{info["url"]}' target='_blank'>{ info["to"] }</a></li>"""
html += "</ol>"

HTML(html)

You can click one by one these links that will prefill you email.  
Or you can use the automatic opening

# Automatic Email Sending Launching

**You need to open this notebook in Jupyter and allow popup in you browser**

It will open a new tab automatically once the previous one is closed.

In [ ]:
display(Javascript("""
let fbAuthWindow;
let urls = {URLs}

function loop_tab() {

    if (!urls.length) {
        alert("You have send all the email !");
        i_clicked_link = 0;
        return
    }
    
    fbAuthWindow = window.open(urls.pop());

    const checkAuthWindow = () => {
        if (fbAuthWindow.closed) {
            loop_tab();
        } else {
            setTimeout(checkAuthWindow, 500);
        }
    };
    checkAuthWindow();
}

loop_tab();
        
""".replace("{URLs}", str([
    info["url"]
    for info in email_info
]))))

# Support

<a href="https://www.buymeacoffee.com/louisgeisler"><img src="https://img.buymeacoffee.com/button-api/?text=Buy me a hot chocolat&emoji=&slug=louisgeisler&button_colour=FFDD00&font_colour=000000&font_family=Lato&outline_colour=000000&coffee_colour=ffffff" /></a>